# Linear Regression

首先把需要用到的包都引入

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [ ]:
df = pd.read_csv('Data.csv')
df = df.drop(['ID'], axis=1)

Y = 0.01 * df['MEDV']
X = 0.01 * df.drop(['MEDV'], axis=1)

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.3, shuffle=True)


## 实验主体：

实现一个 `LinearRegression` 类，具体要求见 `README.md`

In [ ]:
"""
start Lab 01
"""

## 测试

当然你可以更改这部分代码，写自己的测试程序，但务必完成实验中的要求

In [ ]:
lr = LinearRegression()
lr.fit(train_X, train_Y)

x = [i for i in range(lr.epoch)]

plt.plot(x, lr.loss)
plt.show()

"""
.......
"""